## Evaluation of the trained embeddings

Evaluation of the word embeddings. This is the procedure used for the results reported in the paper below.

Authors: 
- F.A. Cardillo, francoalberto.cardillo@cnr.it
- F. Debole, franca.debole@isti.cnr.it

Date: 22 March 2024



__If you use this notebook or the resources it builds, please cite:__

__"Italian Word Embeddings for the Medical Domain", F.A. Cardillo, F. Debole. Proc. of the 2024 Joint Int. Conf. on Computational Linguistics, Language Resources and Evaluation (LREC-COLING 2024), Turin, Italy, May 20-25, 2024.__


The MIT License

Copyright 2024, Franco Alberto Cardillo, Franca Debole

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
<hr>

In [1]:
import pandas as pd
import numpy as np
from posixpath import join
import os
import re

regexp_model = r"(?P<model>\w+)-dim_(?P<vector_size>\d+)-e_(?P<epoch>\d+)-seed_(\d+)-sg_(?P<sg>\d)-w_(?P<window>\d+)-n(?P<negative>\d+)-proc_?-?(?P<n_cores>\d+)\.model"
regext_wv = r"(?P<model>\w+)-dim_(?P<vector_size>\d+)-e_(\d+)-seed_(\d+)-sg_(?P<sg>\d)-w_(?P<window>\d+)-n(?P<negative>\d+)-proc_?-?(?P<n_cores>\d+)__e(?P<epoch>\d+)\.wv"
def split_filename(filename):
    m = re.search(regexp_model, filename)
    if m:
        res = m.groupdict()
        return res
    # test wv
    m = re.search(regext_wv, filename)
    if m:
        res = m.groupdict()
        return res
    assert False, "could not split filename"


aux_fld = join("out", "aux_files")
res_fld = join("out", "eval_resources")
emb_fld = "word-embeddings"

out_fld = join("out", "results")

embs = []
# find all files in emb_fld (and subfolders) with extension .model or 
for root, _, files in os.walk(emb_fld):
    for file in files:
        if file.endswith(".model") or file.endswith(".wv"):
            relative_path = os.path.relpath(root, emb_fld)
            embs.append({"path": relative_path, "filename": file, **split_filename(file)})

emb_df = pd.DataFrame(embs).sort_values(by=["model", "vector_size", "sg", "epoch"])
int_columns = ["vector_size","epoch", "sg", "window", "negative", "n_cores"]
emb_df[int_columns] = emb_df[int_columns].astype(int)
display(emb_df.groupby(["path"]).count())
display(emb_df.groupby(["sg"]).count())

sort_cols = ["model", "sg", "vector_size", "epoch", "window", "negative", "path", "filename"]
emb_df = emb_df.sort_values(by=sort_cols, ascending=True)
emb_df["is_model"] = emb_df.filename.str.endswith(".model")

# # keep one file per group_cols, preferring the model
# group_cols = ["model", "sg", "vector_size", "epoch", "window", "negative"]
# selected_embs = emb_df.groupby(group_cols).apply(lambda g: g.sort_values(by="is_model", ascending=False).head(1))

selected_embs = emb_df.reset_index(drop=True)
print("FOUND EMBEDDINGS")
display(selected_embs)
fn = join(aux_fld, "embeddings.csv")
selected_embs.to_csv(fn, index=False)
print(f"saved to {fn}")


,filename,model,vector_size,epoch,sg,window,negative,n_cores
path,,,,,,,,
pearson,8,8,8,8,8,8,8,8
spearman,8,8,8,8,8,8,8,8


,path,filename,model,vector_size,epoch,window,negative,n_cores
sg,,,,,,,,
0,10,10,10,10,10,10,10,10
1,6,6,6,6,6,6,6,6


FOUND EMBEDDINGS


,path,filename,model,vector_size,epoch,sg,window,negative,n_cores,is_model
0,spearman,w2v-dim_25-e_30-seed_2-sg_0-w_10-n1-proc_4__e1.wv,w2v,25,1,0,10,1,4,False
1,spearman,w2v-dim_25-e_30-seed_2-sg_0-w_10-n1-proc_4__e2.wv,w2v,25,2,0,10,1,4,False
2,pearson,w2v-dim_100-e_30-seed_2-sg_0-w_15-n1-proc_4__e...,w2v,100,1,0,15,1,4,False
3,spearman,w2v-dim_100-e_30-seed_2-sg_0-w_10-n1-proc_4__e...,w2v,100,2,0,10,1,4,False
4,pearson,w2v-dim_100-e_30-seed_2-sg_0-w_15-n5-proc_4__e...,w2v,100,5,0,15,5,4,False
5,spearman,w2v-dim_100-e_30-seed_2-sg_0-w_15-n5-proc_4__e...,w2v,100,5,0,15,5,4,False
6,spearman,w2v-dim_100-e_30-seed_2-sg_0-w_10-n5-proc_4__e...,w2v,100,10,0,10,5,4,False
7,pearson,w2v-dim_100-e_30-seed_2-sg_0-w_30-n5-proc_4__e...,w2v,100,10,0,30,5,4,False
8,pearson,w2v-dim_100-e_30-seed_2-sg_0-w_10-n5-proc_4__e...,w2v,100,25,0,10,5,4,False
9,pearson,w2v-dim_100-e_30-seed_2-sg_0-w_15-n5-proc_4.model,w2v,100,30,0,15,5,4,True


saved to out/aux_files/embeddings.csv


Read `out/aux_files/embeddings.csv`, evaluate the embeddings on `out/eval_resources/sim_rel_all.csv`.

In [2]:
import pandas as pd
import numpy as np
from posixpath import join
import os
import re
from scipy.stats import pearsonr, spearmanr
from gensim.models import Word2Vec, FastText, KeyedVectors


def similarity(v1, v2):
    out = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    # no nans in out
    return np.nan if np.isnan(out).any() else out
#<

missed = 0
def get_vector(wv, term, agg=np.mean):
    global missed
    # print(term)         
    tokens = term.split()
    vectors = []
    
    for t in tokens:
        if t in wv:
            vectors.append(wv[t])
        else:
            missed = missed + 1
        
    res = agg(vectors, axis=0) if len(vectors) > 0 else np.nan
    if type(res) == np.ndarray:
        assert not np.isnan(res).any()
    return res
#<

def compute_correlation(group, path, model_name):
    group2 = group.dropna(subset=["computed_score", "score"], how="any")
    values1 = np.array(group2.computed_score)
    values2 = np.array(group2.score)
    r, p = pearsonr(values1, values2)
    group["r"] = r
    group["p"] = p
    sr, sp = spearmanr(values1, values2)
    group["spearman"] = sr
    group["spearman_p"] = sp

    group["path"] = path
    group["model_name"] = model_name
    return group

def compute_all(model, terms_df, path, model_name):
    data = terms_df.copy()
    #print("COMPUTE ALL")
    #display(data)
    vectors1 = data.term1_it.apply(lambda x: get_vector(model, x))
    vectors2 = data.term2_it.apply(lambda x: get_vector(model, x))
    data["v1"] = vectors1
    data["v2"] = vectors2
    sim = []
    for v1, v2 in zip(vectors1, vectors2):
        try:
            s = similarity(v1, v2)
        except ValueError:
            s = np.nan
        sim.append(s)
    #< for over vectors

    data["computed_score"] = sim
    data["size"] = data.groupby(["filename", "annotator"])["term1_it"].transform('count')
    data["valid"] = data.v1.apply(lambda x: not np.isnan(x).any()) & data.v2.apply(lambda x: not np.isnan(x).any())
    data["n_valid"] = data.groupby(["filename", "annotator"])["valid"].transform('sum')

    # optimization here: should be data, but we can use df and it is faster
    out = data.groupby(["filename", "annotator"]).apply(lambda g: compute_correlation(g, path, model_name)).reset_index(drop=True)
    out["stat_significant"] = out.p < 0.05
    out["spear_stat_significant"] = out.spearman_p < 0.05
    return out
    
#<

def process_row(row, terms_df):
    path = join(base_emb_fld, row.path, row.filename)
    mt = row.model
    print(f"loading {mt} from: {path}")
    is_model = row.filename.endswith(".model")
    if is_model:
        model = Word2Vec.load(path) if mt == "w2v" else FastText.load(path)
        model = model.wv
    else:
        model = KeyedVectors.load(path)
    out = compute_all(model, terms_df, row.path, row.filename)
    out["sg"] = row.sg
    out["vector_size"] = row.vector_size 
    out["epoch"] = row.epoch
    out["window"]= row.window 
    out["negative"] = row.negative
    #cols = ["filename", "annotator", "path", "model_name", "stat_significant", "r", "p", "spearman", "spearman_p", "spear_stat_significant", "size", "n_valid"]
    #display(out[cols].groupby(["filename", "annotator"]).head(1))
    all_outs.append(out[["filename", "annotator", "sg", "path", "model_name", "stat_significant", "r", "p", "spearman", "spearman_p", "spear_stat_significant", "size", "n_valid", "vector_size",	"epoch", "window", "negative"]])

base_emb_fld = "word-embeddings"

aux_fld = join("out", "aux_files")
emb_df = pd.read_csv(join(aux_fld, "embeddings.csv"))
print("EMBEDDINGS")
display(emb_df)

df = pd.read_csv(join(res_fld, "sim_rel_all.csv"))
# remove resources with "word2vec" in filename
df = df[~df.filename.str.contains("word2vec")]
print("SIM_REL_ALL.CSV")
display(df.groupby(["filename", "annotator"]).sample(1))

all_outs = []
emb_df.apply(lambda r: process_row(r, df), axis=1)

results = pd.concat(all_outs)
print("all results, shape:", results.shape)

best_cols = ["r", "stat_significant",  "vector_size",	"epoch", "window", "negative"]
pearson_best = results.groupby(["filename", "annotator", "sg"]).apply(lambda g: g.sort_values(by=["r"], ascending=False).head(1))
print("PEARSON results, best, shape:", pearson_best.shape)
display(pearson_best[best_cols])

best_cols = ["spearman", "spear_stat_significant",  "vector_size",	"epoch", "window", "negative"]
spearman_best = results.groupby(["filename", "annotator", "sg"]).apply(lambda g: g.sort_values(by=["spearman"], ascending=False).head(1))
print("SPEARMAN results, best, shape:", spearman_best.shape)
display(spearman_best[best_cols])


out_fld = join("out", "results")
os.makedirs(out_fld, exist_ok=True)

fn = join(out_fld, "best_results_lrec24_pearson.csv")
pearson_best.to_csv(fn, index=False)
print(f"saved to {fn}")

fn = join(out_fld, "best_results_lrec24_spearman.csv")
spearman_best.to_csv(fn, index=False)
print(f"saved to {fn}")

print("all done")


EMBEDDINGS


,path,filename,model,vector_size,epoch,sg,window,negative,n_cores,is_model
0,spearman,w2v-dim_25-e_30-seed_2-sg_0-w_10-n1-proc_4__e1.wv,w2v,25,1,0,10,1,4,False
1,spearman,w2v-dim_25-e_30-seed_2-sg_0-w_10-n1-proc_4__e2.wv,w2v,25,2,0,10,1,4,False
2,pearson,w2v-dim_100-e_30-seed_2-sg_0-w_15-n1-proc_4__e...,w2v,100,1,0,15,1,4,False
3,spearman,w2v-dim_100-e_30-seed_2-sg_0-w_10-n1-proc_4__e...,w2v,100,2,0,10,1,4,False
4,pearson,w2v-dim_100-e_30-seed_2-sg_0-w_15-n5-proc_4__e...,w2v,100,5,0,15,5,4,False
5,spearman,w2v-dim_100-e_30-seed_2-sg_0-w_15-n5-proc_4__e...,w2v,100,5,0,15,5,4,False
6,spearman,w2v-dim_100-e_30-seed_2-sg_0-w_10-n5-proc_4__e...,w2v,100,10,0,10,5,4,False
7,pearson,w2v-dim_100-e_30-seed_2-sg_0-w_30-n5-proc_4__e...,w2v,100,10,0,30,5,4,False
8,pearson,w2v-dim_100-e_30-seed_2-sg_0-w_10-n5-proc_4__e...,w2v,100,25,0,10,5,4,False
9,pearson,w2v-dim_100-e_30-seed_2-sg_0-w_15-n5-proc_4.model,w2v,100,30,0,15,5,4,True


SIM_REL_ALL.CSV


,filename,cui1,cui2,annotator,term1,term2,score,term1_it,term2_it
6,MayoSRS_it.csv,C0085649,C0034063,coders,peripheral edema,pulmonary edema,3.92,edema periferico,edema polmonare
157,MiniMayoSRS_it.csv,C0042345,C0224701,coders,Varicose vein,Entire knee meniscus,1.00,vene varicose,menisco del ginocchio intero
127,MiniMayoSRS_it.csv,C0034887,C0003483,physicians,Rectal polyp,Aorta,1.00,polipo rettale,aorta
634,UMNSRS_relatedness_it.csv,C0030232,C0001924,umnrs,Pallor,Albumin,987.50,pallore,albumine
1531,UMNSRS_similarity_it.csv,C0152149,C0003564,umnrs,Mittelschmerz,Aphonia,199.25,dolore dell'ovulazione,afonia


loading w2v from: word-embeddings/spearman/w2v-dim_25-e_30-seed_2-sg_0-w_10-n1-proc_4__e1.wv
loading w2v from: word-embeddings/spearman/w2v-dim_25-e_30-seed_2-sg_0-w_10-n1-proc_4__e2.wv
loading w2v from: word-embeddings/pearson/w2v-dim_100-e_30-seed_2-sg_0-w_15-n1-proc_4__e1.wv
loading w2v from: word-embeddings/spearman/w2v-dim_100-e_30-seed_2-sg_0-w_10-n1-proc_4__e2.wv
loading w2v from: word-embeddings/pearson/w2v-dim_100-e_30-seed_2-sg_0-w_15-n5-proc_4__e5.wv
loading w2v from: word-embeddings/spearman/w2v-dim_100-e_30-seed_2-sg_0-w_15-n5-proc_4__e5.wv
loading w2v from: word-embeddings/spearman/w2v-dim_100-e_30-seed_2-sg_0-w_10-n5-proc_4__e10.wv
loading w2v from: word-embeddings/pearson/w2v-dim_100-e_30-seed_2-sg_0-w_30-n5-proc_4__e10.wv
loading w2v from: word-embeddings/pearson/w2v-dim_100-e_30-seed_2-sg_0-w_10-n5-proc_4__e25.wv
loading w2v from: word-embeddings/pearson/w2v-dim_100-e_30-seed_2-sg_0-w_15-n5-proc_4.model
loading w2v from: word-embeddings/pearson/w2v-dim_50-e_25-seed_1-

r  stat_significant  \
filename                  annotator  sg                                   
MayoSRS_it.csv            coders     0  76   0.492332              True   
                                     1  100  0.571283              True   
MiniMayoSRS_it.csv        coders     0  122  0.822083              True   
                                     1  129  0.779522              True   
                          physicians 0  158  0.748259              True   
                                     1  158  0.779303              True   
UMNSRS_relatedness_it.csv umnrs      0  159  0.475774              True   
                                     1  159  0.488323              True   
UMNSRS_similarity_it.csv  umnrs      0  747  0.592940              True   
                                     1  887  0.595121              True   

                                             vector_size  epoch  window  \
filename                  annotator  sg                                   
MayoSRS_it.csv            coders     0  76           100     10      30   
                                     1  100          100     30      30   
MiniMayoSRS_it.csv        coders     0  122          100      1      15   
                                     1  129          100     30      30   
                          physicians 0  158          100     30      15   
                                     1  158          100     30      30   
UMNSRS_relatedness_it.csv umnrs      0  159          100      5      15   
                                     1  159           50     25       5   
UMNSRS_similarity_it.csv  umnrs      0  747          100     25      10   
                                     1  887          100     30      10   

                                             negative  
filename                  annotator  sg                
MayoSRS_it.csv            coders     0  76          5  
                                     1  100         1  
MiniMayoSRS_it.csv        coders     0  122         1  
                                     1  129         1  
                          physicians 0  158         5  
                                     1  158         1  
UMNSRS_relatedness_it.csv umnrs      0  159         5  
                                     1  159         5  
UMNSRS_similarity_it.csv  umnrs      0  747         5  
                                     1  887         1

SPEARMAN results, best, shape: (10, 17)


spearman  \
filename                  annotator  sg                  
MayoSRS_it.csv            coders     0  2     0.527977   
                                     1  100   0.578107   
MiniMayoSRS_it.csv        coders     0  101   0.804123   
                                     1  129   0.835905   
                          physicians 0  144   0.740491   
                                     1  158   0.803554   
UMNSRS_relatedness_it.csv umnrs      0  159   0.506851   
                                     1  305   0.500779   
UMNSRS_similarity_it.csv  umnrs      0  1171  0.602876   
                                     1  746   0.595249   

                                              spear_stat_significant  \
filename                  annotator  sg                                
MayoSRS_it.csv            coders     0  2                       True   
                                     1  100                     True   
MiniMayoSRS_it.csv        coders     0  101                     True   
                                     1  129                     True   
                          physicians 0  144                     True   
                                     1  158                     True   
UMNSRS_relatedness_it.csv umnrs      0  159                     True   
                                     1  305                     True   
UMNSRS_similarity_it.csv  umnrs      0  1171                    True   
                                     1  746                     True   

                                              vector_size  epoch  window  \
filename                  annotator  sg                                    
MayoSRS_it.csv            coders     0  2             100      2      10   
                                     1  100           100     30      30   
MiniMayoSRS_it.csv        coders     0  101            25      1      10   
                                     1  129           100     30      30   
                          physicians 0  144            25      2      10   
                                     1  158           100     30      30   
UMNSRS_relatedness_it.csv umnrs      0  159           100      5      15   
                                     1  305           100     30      10   
UMNSRS_similarity_it.csv  umnrs      0  1171          100     10      10   
                                     1  746           100     30      15   

                                              negative  
filename                  annotator  sg                 
MayoSRS_it.csv            coders     0  2            1  
                                     1  100          1  
MiniMayoSRS_it.csv        coders     0  101          1  
                                     1  129          1  
                          physicians 0  144          1  
                                     1  158          1  
UMNSRS_relatedness_it.csv umnrs      0  159          5  
                                     1  305          1  
UMNSRS_similarity_it.csv  umnrs      0  1171         5  
                                     1  746          1

saved to out/results/best_results_lrec24_pearson.csv
saved to out/results/best_results_lrec24_spearman.csv
all done
